# Social-Network Query Exercises + Extras  
***

## Schema  

### Highschooler (ID, name, grade)  
There is a high school student with unique ID and a given first name in a certain grade.  
### Friend (ID1, ID2)  
The student with ID1 is friends with the student with ID2. Friendship is mutual, so if (123, 456) is in the Friend table, so is (456, 123).  
### Likes (ID1, ID2)  
The student with ID1 likes the student with ID2. Liking someone is not necessarily mutual, so if (123, 456) is in the likes table, there is no guarantee that (456, 123) is also present.  
***

#### Q1: Find the name of all students who are friends with someone named Gabriel.

In [ ]:
SELECT h.name
FROM Highschooler h
JOIN Friend f ON h.ID = f.ID1
WHERE f.ID2 IN (SELECT h1.ID FROM Highschooler h1 WHERE h1.name = 'Gabriel');

#### Q2: For every student who likes someone 2 or more grades younger than themselves, return that student's name and grade, and the name and grade of the student they like.

In [ ]:
SELECT h1.name, h1.grade, h2.name, h2.grade
FROM Highschooler h1
JOIN Likes l ON h1.ID = l.ID1
JOIN Highschooler h2 ON h2.ID = l.ID2
WHERE (h1.grade - h2.grade) >= 2;

#### Q3: For every pair of students who both like each other, return the name and grade of both students. Include each pair only once, with the two names in alphabetical order.

In [ ]:
SELECT h1.name, h1.grade, h2.name, h2.grade
FROM Likes l1
JOIN Likes l2 ON l1.ID2 = l2.ID1 AND l2.ID2 = l1.ID1
JOIN Highschooler h1 ON l1.ID1 = h1.ID
JOIN Highschooler h2 ON l1.ID2 = h2.ID
WHERE h1.name < h2.name;

#### Q4: Find all students who do not appear in the Likes table ( as a student who likes or is liked) and return their names and grades. Sort by grade, then by name within each grade.

In [ ]:
SELECT h.name, h.grade
FROM Highschooler h
WHERE h.ID NOT IN (SELECT ID1 FROM Likes
                    UNION
                   SELECT ID2 FROM Likes);

#### Q5: For every situation where student A likes student B, but we have no information about whom B likes (that is, B does not appear as an ID1 in the Likes table), return A and B's names and grades.

In [ ]:
SELECT h1.name, h1.grade, h2.name, h2.grade
FROM Highschooler h1
JOIN Likes l ON h1.ID = l.ID1
JOIN Highschooler h2 ON h2.ID = l.ID2
WHERE l.ID2 NOT IN (SELECT l1.ID1 FROM Likes l1);

#### Q6: Find names and grades of students who only have friends in the same grade. Return the result sorted by grade, then by name within each grade.

In [ ]:
SELECT h1.name, h1.grade
FROM Friend f1
JOIN Highschooler h1 ON f1.ID1 = h1.ID
JOIN Highschooler h2 ON f1.ID2 = h2.ID
GROUP BY f1.ID1
HAVING MAX(h1.grade <> h2.grade) = 0
ORDER BY h1.grade, h1.name;

#### Q7: For each student A who likes a student B where the two are not friends, find if they have a friend C in common (who can introduce them!). For all such trios, return the name and grade of A, B, and C.

In [ ]:
SELECT h1.name, h1.grade, h2.name, h2.grade, h3.name, h3.grade
FROM Highschooler h1, Highschooler h2, Highschooler h3, Friend f1, Friend f2,
        (SELECT l.ID1, l.ID2 FROM Likes l
        EXCEPT
        SELECT l.ID1, l.ID2
        FROM Likes l, Friend as f
        WHERE l.ID1 = f.ID1 AND l.ID2 = f.ID2) d
WHERE h1.ID = f1.ID1 and f1.ID1 = d.ID1
AND h2.ID = f2.ID1 and f2.ID1 = d.ID2 
AND h3.ID = f2.ID2 and f2.ID2 = f1.ID2;

#### Q8: Find the difference between the number of students in the school and the number of different first names.

In [ ]:
SELECT COUNT(*) - (SELECT (COUNT(DISTINCT name) FROM Highschooler)
FROM Highschooler;

#### Q9: Find the name and grade of all students who are liked by more than one other student.

In [ ]:
SELECT h.name, h.grade
FROM Highschooler h,
    (SELECT l.ID2 AS ID
     FROM Likes l
     GROUP BY l.ID2
     HAVING COUNT(l.ID2) > 1) f
WHERE h.ID = f.ID;

## Extras
***

#### Q1: For every situation where student A likes student B, but student B likes a different student C, return the names and grades of A, B, and C.

In [ ]:
SELECT h1.name, h1.grade, h2.name, h2.grade, h3.name, h3.grade
FROM Highschooler h1
JOIN Likes l1 ON h1.ID = l1.ID1
JOIN Highschooler h2 ON l1.ID2 = h2.ID
JOIN Likes l2 ON h2.ID = l2.ID1
JOIN Highschooler h3 ON h3.ID = l2.ID2
WHERE h1.ID <> h3.ID;

#### Q2: Find those students for whom all of their friends are in different grades from themselves. Return the students' names and grades.

In [ ]:
SELECT DISTINCT h1.name, h1.grade
FROM Highschooler h1
JOIN Friend f1 ON h1.ID = f1.ID1
WHERE h1.grade NOT IN
    (SELECT h2.grade
     FROM Highschooler h2
     JOIN Friend f2
     WHERE h1.ID = f2.ID1
     AND h2.ID = f2.ID2);

#### Q3: What is the average number of friends per student?

In [ ]:
SELECT AVG(new.c)
FROM (SELECT COUNT(f.ID2) as c
                FROM Highschooler h JOIN Friend f ON h.ID = f.ID1
                GROUP BY f.ID1) new;

#### Q4: Find the number of students who are either friends with Cassandra or are friends of friends of Cassandra. Do not count Cassandra, even though technically she is a friend of a friend.

In [ ]:
SELECT COUNT(DISTINCT f1.ID2)
FROM friend f1
WHERE f1.ID1 =
        (SELECT id
        FROM Highschooler
        WHERE name = 'Cassandra')
OR f1.ID1 IN
(SELECT f2.ID2
    FROM friend AS f2
    WHERE f2.ID1 = 
        (SELECT ID
        FROM Highschooler
        WHERE name = 'Cassandra'))
AND f1.ID2 !=
    (SELECT ID
     FROM Highschooler
     WHERE name = 'Cassandra');

#### Q5: Find the name and grade of the student(s) with the greatest number of friends.

In [ ]:
SELECT h.name, h.grade
FROM Highschooler h
JOIN Friend f1 ON h1.ID = f1.ID1
GROUP BY h.ID
HAVING COUNT(f1.ID2) =
        (SELECT MAX(t.fcount)
        FROM
            (SELECT f2.ID1, COUNT(f2.ID2) AS fcount
            FROM Friend f2
            GROUP BY f2.ID1) AS t);